In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# Problem1

In [2]:
def create_gaussian_kernel(k, s):
    """
    Create a 2D Gaussian kernel.
    """
    ax = np.arange(-k // 2 + 1., k // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * s**2))
    return kernel / np.sum(kernel)

In [3]:
def apply_convolution(image, kernel):
    """
    Apply convolution to the image using the given kernel.
    """
    m, n = kernel.shape
    y, x = image.shape
    y = y - m + 1
    x = x - n + 1
    output = np.zeros((y,x))
    for i in range(y):
        for j in range(x):
            output[i,j] = np.sum(image[i:i+m, j:j+n]*kernel)
    return output

In [4]:
def gradient_edge_detector(image, sigma):
    """
    Perform gradient-based edge detection as per PDF instructions.
    """
    # Create Gaussian kernel
    kernel_size = int(6 * sigma + 1)  # Ensure kernel size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    gaussian_kernel = create_gaussian_kernel(kernel_size, sigma)
    
    # Smooth the image with a Gaussian filter
    smoothed = apply_convolution(image, gaussian_kernel)
    
    # Compute horizontal and vertical derivatives
    dx_kernel = np.array([[-1, 0, 1]])
    dy_kernel = np.array([[-1], [0], [1]])
    
    grad_x = apply_convolution(smoothed, dx_kernel)
    grad_y = apply_convolution(smoothed, dy_kernel)
    
    # Ensure grad_x and grad_y have the same shape
    min_shape = (min(grad_x.shape[0], grad_y.shape[0]), min(grad_x.shape[1], grad_y.shape[1]))
    grad_x = grad_x[:min_shape[0], :min_shape[1]]
    grad_y = grad_y[:min_shape[0], :min_shape[1]]
    
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    orientation = np.arctan2(grad_y, grad_x)
    
    return magnitude, orientation

In [5]:
def plot_results(image, magnitude, orientation):
    """
    Plot the original image, gradient magnitude, and gradient orientation.
    """
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(magnitude, cmap='gray')
    plt.title('Gradient Magnitude')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    step = 10  # Adjust this to change the density of arrows
    y, x = np.mgrid[step//2:magnitude.shape[0]:step, step//2:magnitude.shape[1]:step]
    u = np.cos(orientation[y, x])
    v = np.sin(orientation[y, x])
    plt.quiver(x, y, u, v, color='r', angles='xy', scale_units='xy', scale=0.1)
    plt.title('Gradient Orientation')
    plt.gca().invert_yaxis()
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [6]:
#Load and preprocess the image
image = np.array(Image.open('./1.png').convert('L'))
image = image.astype(np.float32) / 255.0  # Normalize to [0, 1]

# Set parameter
sigma = 1.0

# Apply gradient-based edge detection
magnitude, orientation = gradient_edge_detector(image, sigma)

In [ ]:
# Plot results
plot_results(image, magnitude, orientation)

In [ ]:
# You can experiment with different sigma values here
sigmas = [0.5, 1.0, 2.0]
for sigma in sigmas:
    magnitude, orientation = gradient_edge_detector(image, sigma)
    plot_results(image, magnitude, orientation)
    print(f"Results for sigma = {sigma}")